In [6]:
import numpy as np
from numpy.linalg import multi_dot
from numpy.linalg import qr
import numpy.linalg as npla
import scipy as sp
from scipy.linalg import block_diag, logm, eigvals
from multiprocessing import Pool
from functools import partial
import multiprocessing as mp
from multiprocessing import set_start_method
from multiprocessing import get_context
import matplotlib.pyplot as plt
import time
from scipy.linalg import lu
'''
All the matrix construction functions are defined in this file, they include

TAMatrix
TBMatrix
TAReplace
TBReplace

All functions take arguments (sin,cos)
They are all 2x2 np.arrays
'''
from matrix_constructions import *

In [7]:
#This function constructs full strips with width = strip_width
def FullStrip(sin,cos,strip_width,probability_cutoff): #np.array 2strip_width x 2strip_width
    
        #we construct TAS for TA type strips and multiply by TB type strips
        #this means our length is really 2x our strip_length followig the convention of CC
        #probability cutoff is first introduced here to replace specific nodes in both types of strips
        
        def TAS(sin,cos,strip_width,probability_cutoff):
            
            TAProbabilities = np.random.random_sample(strip_width)
            
            Tslist = [ 
                TAMatrix(sin,cos) if i < probability_cutoff 
                else TAReplace(sin,cos) for i in TAProbabilities
            ]
            
            return block_diag(*Tslist)

        
        def TBS(sin,cos,strip_width,probability_cutoff): 
            
            TBProbabilities = np.random.random_sample(strip_width-1)
            
            Tslist = [ 
                TBMatrix(sin,cos)if i < probability_cutoff
                else TBReplace(sin,cos) for i in TBProbabilities
            ]
            
            extra = TBMatrix(sin,cos)
            temp_mat = block_diag(extra[1,1],*Tslist,extra[0,0])    
            temp_mat[0,(2*strip_width)-1] = extra[1,0]
            temp_mat[(2*strip_width)-1,0] = extra[0,1]
            return temp_mat
    
        return np.matmul(TAS(sin,cos,strip_width,probability_cutoff),
                         TBS(sin,cos,strip_width,probability_cutoff))



In [8]:
#Full Transfer multiples a bunch of strips together
#This is where the QR/LU is performed to actaully do the computation
#This is the last function that is run

def FullTransfer(strip_length,strip_width,probability_cutoff, theta): #Returns np.array with length 2strip_width 
    #Saving values of Sin and Cos
    sin = np.sin(theta)
    cos = np.cos(theta)
    
    #Group_val describes how often the LU is carried out
    group_val = 8
    
    #creating matricies
    matricies = [FullStrip(sin,cos,strip_width,probability_cutoff) for i in range(strip_length)]
    #splitting matricies
    every_nth = [multi_dot(matricies[i:i+group_val]) for i in range(int(strip_length/group_val))]
    
    #This step is proved by induction, find in literature
    Tone = matricies[0]
    pone,lone,uone = lu(Tone)
    bigQ = np.matmul(pone,lone)
    rlog_one = np.log(np.absolute(uone.diagonal()))
    
    for n,i in enumerate(every_nth):
        matrixb = np.matmul(i,bigQ)
        p,l,u = lu(matrixb)
        bigQ = np.matmul(p,l)
        rlogs = np.log(np.absolute(u.diagonal()))
        rlog_one = np.add(rlogs,rlog_one)
        #autosave every 50000
        if ( n % 50000):
            np.save('matrix_logs/LULogQ.npy', bigQ)
            np.save('matrix_logs/LULogR.npy',rlog_one)
    return (rlog_one)

In [ ]:
strip_length = int(10000)
#strip_width here is actually 2x what the "physical" result is but will give a matrix that is 2x the specified size
strip_width = int(16)

#probability cutoff specified here
probability_cutoff = 1.0

#energy is set here
theta_crit =np.pi/4

start_time = time.time()

#theta values
theta_list = np.linspace(theta_crit,1.5,15)

partial_func = partial(FullTransfer,strip_length,strip_width,probability_cutoff)
p = get_context("fork").Pool()

#using multiple cores here
final_array= p.map(partial_func, theta_list)


np.save('results/length1000_width8_pi4_probhalf.npy',final_array)
print("--- %s minutes---" % ((time.time() - start_time)/60))

In [ ]:
plt.scatter([i for i in range(strip_width * 2)],final_array[1])


In [13]:
len(final_array[0])

32

In [18]:
len(test)

32